### Environment setup

In [2]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen

### HTML parsing

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
pag = urlopen(url).read().decode('utf-8')
sou = bs(pag, 'html.parser')
tab = sou.body.table.tbody

In [4]:
tab_tr = str(tab).split('<tr>')
tab_cod = pd.DataFrame(columns = ['cod','bor','nei'])
for i in range(1, len(tab_tr)):
    tab_td = tab_tr[i].split('/td')
    for j in range(len(tab_td)):
        if '<td>' in tab_td[j]:
            a = tab_td[j].split('<td>')[1]
            b = tab_td[j + 1].split('<td>')[1]
            c = tab_td[j + 2].split('<td>')[1]
            if ('<a href=' in b) or ('<a class=' in b):
                b = b.split('</a>')[0].split('">')[1]
            if ('<a href=' in c) or ('<a class=' in c):
                c = c.split('</a>')[0].split('">')[1]
            tab_cod.loc[len(tab_cod)] = [a.replace('<',''),b.replace('<','').replace('\n',''),c.replace('<','').replace('\n','')]
            break

In [5]:
tab_cod.head()

,cod,bor,nei
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Data cleaning

I first count the values in Borough

In [6]:
tab_cod['bor'].value_counts()

Not assigned        77
Etobicoke           45
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Queen's Park         1
Mississauga          1
Name: bor, dtype: int64

I ignore the cells with a borough that is 'Not assigned'.

In [7]:
tab_cle = tab_cod[tab_cod['bor'] != 'Not assigned']
tab_cle['bor'].value_counts()

Etobicoke           45
North York          38
Scarborough         37
Downtown Toronto    37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Queen's Park         1
Mississauga          1
Name: bor, dtype: int64

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [8]:
tab_cle[tab_cle['nei'] == 'Not assigned'].head()

,cod,bor,nei
8,M7A,Queen's Park,Not assigned


In [9]:
tab_cle['nei'][tab_cle['nei'] == 'Not assigned'] = tab_cle['bor']

c:\users\pc\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:7626: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
c:\users\pc\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


I confirm that there are not 'Not assigned' neighbourhoods

In [10]:
tab_cle[tab_cle['nei'] == 'Not assigned'].head()

,cod,bor,nei


More than one neighborhood can exist in one postal code area.

In [11]:
tab_cle.head(20)

,cod,bor,nei
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


I create a new df in which a PostalCode has only one row. I assign the 'Borough' and 'Neiborhood' values according to the PostalCode value.

In [12]:
lis_cod = tab_cle['cod'].value_counts().index

tab_fin = pd.DataFrame(columns = ['PostalCode','Borough','Neighborhood'])

for i in range(len(lis_cod)):
    tab_fin.loc[len(tab_fin)] = [lis_cod[i],list(tab_cle['bor'][tab_cle['cod'] == lis_cod[i]])[0],', '.join(list(tab_cle['nei'][tab_cle['cod'] == lis_cod[i]]))]

In [13]:
tab_fin.sort_values(['PostalCode'], inplace=True)

In [14]:
tab_fin.head()

,PostalCode,Borough,Neighborhood
44,M1B,Scarborough,"Rouge, Malvern"
12,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
10,M1E,Scarborough,"Guildwood, Morningside, West Hill"
100,M1G,Scarborough,Woburn
96,M1H,Scarborough,Cedarbrae


In [15]:
tab_fin.to_csv('tab_fin.csv', index = False)

In [16]:
tab_fin.shape

(103, 3)